In [1]:
import pandas as pd
from torch.utils.data import DataLoader
import pytorch_lightning as pl
#from pytorch_lightning.profiler import Profiler, AdvancedProfiler
import HeatfluxData as hfd
import HeatfluxModel as hfm

/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename_data = f'../students/bogdaale/NN-examples/scaled_QdataKn25width100microns.csv'
# Using heat flux data module
scaled_Qdata = pd.read_csv(filename_data, header=0, index_col=0)
test_set, train_set, validation_set, vis_set = hfd.heat_flux_datasets(scaled_Qdata, 0.3, 0.8)

In [3]:
### Create data loaders
train_loader = DataLoader(dataset = train_set, batch_size = 128)
validation_loader = DataLoader(dataset = validation_set, batch_size = 128)
test_loader = DataLoader(dataset = test_set, batch_size = 128)
# Special object for visualization
vis_loader = DataLoader(dataset = vis_set, batch_size = 128)

In [4]:
### Read data sacling (generated by the data manager)
data_scaling=pd.read_csv(f'../students/bogdaale/NN-examples/data_scaling.csv', header=0, index_col=0)

In [5]:
### Create the heat flux model
Nfeatures = train_set[0][0].size()[0]# TODO: find a better way than extracting the size via Tensor
print(f'Nfeatures {Nfeatures}')
Nlayer1 = 20
Nlayer2 = 10
model = hfm.AlphacModel(Nfeatures, Nlayer1, Nlayer2, data_scaling)

Nfeatures 125


In [6]:
### Let's train the beast!
trainer = pl.Trainer(max_epochs = 10)
trainer.fit(model, train_loader, validation_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/holec1/ML-student-projects/PyTorch/lightning_logs

  | Name     | Type    | Params
-------------------------------------
0 | fc1      | Linear  | 2.5 K 
1 | fc2      | Linear  | 210   
2 | fc3      | Linear  | 11    
3 | mse_loss | MSELoss | 0     
-------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)


/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, d

Epoch 0:  80%|██████████████████████▎     | 166/208 [00:01<00:00, 146.07it/s, loss=nan, v_num=0]

/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Validation: 0it [00:00, ?it/s]
Epoch 1:  13%|███▉                         | 28/208 [00:00<00:01, 147.76it/s, loss=nan, v_num=0]

...

Epoch 7:  80%|██████████████████████▎     | 166/208 [00:01<00:00, 152.13it/s, loss=nan, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 8:  80%|██████████████████████▎     | 166/208 [00:01<00:00, 152.40it/s, loss=nan, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 8:  83%|███████████████████████▏    | 172/208 [00:01<00:00, 153.07it/s, loss=nan, v_num=0]


Epoch 9:  80%|██████████████████████▎     | 166/208 [00:01<00:00, 152.75it/s, loss=nan, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|████████████████████████████| 208/208 [00:01<00:00, 160.40it/s, loss=nan, v_num=0]


Epoch 9: 100%|████████████████████████████| 208/208 [00:01<00:00, 160.14it/s, loss=nan, v_num=0]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|████████████████████████████| 208/208 [00:01<00:00, 159.67it/s, loss=nan, v_num=0]
